In [ ]:
# Load Canary model
from nemo.collections.asr.models import EncDecMultiTaskModel
import os

parkeet_model = EncDecMultiTaskModel.from_pretrained('nvidia/parakeet-ctc-1.1b')

def get_wav_files(directory):
    return [os.path.join(directory, file) for file in os.listdir(directory) if file.endswith('.wav')]

wav_files = sorted(get_wav_files('/home/s44504/3b01c699-3670-469b-801f-13880b9cac56/speechline/bookbot_en/en-ar'))
# Transcribe
predicted_text = parkeet_model.transcribe(paths2audio_files=wav_files, batch_size=16)

In [2]:
predicted_text

['the same was f okay i see okay this is',
 'i see where are we where is it',
 'and',
 'sten st',
 'steve and alex go out to fish they get a lot of cod',
 'it is a big job to get all of the cold heat com',
 'they do not think of good way to store it good way to store it',
 'they put some cood on the fire they meal their meal for their meal i',
 'they put the rest in the house it gets old',
 'soon there is a strong pun in the house it smells of rot',
 'strong',
 'steve puts the rod on the fire the smell his the smell makes his eyes sting',
 'br',
 'five five',
 'then',
 'alex alex put put some rot in a standan for a mob to snatch up']

In [ ]:
# Load Canary model
from nemo.collections.asr.models import EncDecMultiTaskModel
import os
import torch

torch.cuda.is_available = lambda : False

parkeet_model = EncDecMultiTaskModel.from_pretrained('nvidia/canary-1b')

def get_wav_files(directory):
    return [os.path.join(directory, file) for file in os.listdir(directory) if file.endswith('.wav')]

wav_files = sorted(get_wav_files('/home/s44504/3b01c699-3670-469b-801f-13880b9cac56/speechline/bookbot_en/en-ar'))
# Transcribe
predicted_text = parkeet_model.transcribe(paths2audio_files=wav_files, batch_size=16)

In [5]:
predicted_text

['Decide your friend. What? What if what he says?',
 'Pong Pong Pong Pong Pong Pong Pong Pong Pong Pong Pong Pong Pong Pong Pong Pong Pong Pong Pong Pong Pong Pong Pong Pong Pong Pong Pong Pong Pong Pong Pong Pong Pong Pong Pong Pong Pong Pong Pong Pong Pong Pong Pong Pong Pong Pong Pong Pong Pong Pong Pong Pong Pong Pong Pong Pong Pong Pong Pong Pong Pong Pong Pong Pong Pong Pong Pong Pong Pong Pong Pong Pong Pong Pong Pong Pong P',
 'Henry,',
 'Stephen, Stephen, Stephen, Harry.',
 'Steve and Alex go out to fish. They get a lot of cod.',
 'It is a big job to get all of the cold heat home.',
 'They do not think of good way to store it. Good way to store it.',
 'They put some cod on the fire they meal, their meal, for their meal, anyway,',
 'They put the rest in the house, it gets old.',
 'Soon there is a strong pong in the house, it smells of rot.',
 'Strong.',
 'Steve puts the rod on the fire. The smell make his smell make his eyes sting.',
 'Bron.',
 'Fire. Fire.',
 'Come, then.',
 '

In [ ]:
import os
import json

# Directory containing your audio files
audio_dir = "/home/s44504/3b01c699-3670-469b-801f-13880b9cac56/speechline/bookbot_en/en-ar"

# Output manifest file
manifest_path = 'manifest.json'

# Collect all .wav files in the directory
audio_files = [f for f in os.listdir(audio_dir) if f.endswith('.wav')]

# Create the manifest
with open(manifest_path, 'w') as manifest_file:
    for audio_file in audio_files:
        entry = {
            'audio_filepath': os.path.join(audio_dir, audio_file)
        }
        manifest_file.write(json.dumps(entry) + '\n')

print(f'Manifest created at {manifest_path}')

In [ ]:
## Whisper

In [1]:
import torch
import os
import time
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline

device = "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

model_id = "openai/whisper-large-v3"

model = AutoModelForSpeechSeq2Seq.from_pretrained(
model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=True, use_safetensors=True
)
model.to(device)

processor = AutoProcessor.from_pretrained(model_id)

pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    chunk_length_s=30,
    torch_dtype=torch_dtype,
    device=device,
)

generate_kwargs = {
    "num_beams": 1,
    "logprob_threshold": -1.0,
    "no_speech_threshold": 0.6,
    "temperature": 1.0,
}


def get_wav_files(directory):
    return [os.path.join(directory, file) for file in os.listdir(directory) if file.endswith('.wav')]

wav_files = sorted(get_wav_files('/home/s44504/3b01c699-3670-469b-801f-13880b9cac56/speechline/bookbot_en/en-ar'))
wav_files = ["/home/s44504/3b01c699-3670-469b-801f-13880b9cac56/speechline/bookbot_en/en-ar/guest_78702eb3-bcfc-48f3-9f31-e2e2e5733958_1722800307336.wav"]
result = pipe(wav_files, generate_kwargs=generate_kwargs, return_timestamps="word")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Audio input consists of only 3000. Short-form transcription is activated.logprob_threshold is set to -1.0, but will be ignored.
Audio input consists of only 3000. Short-form transcription is activated.no_speech_threshold is set to 0.6, but will be ignored.
Due to a bug fix in https://github.com/huggingface/transformers/pull/28687 transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English.This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`.
WhisperModel is using WhisperSdpaAttention, but `torch.nn.functional.scaled_dot_product_attention` does not support `output_attentions=True` or `layer_head_mask` not None. Falling back to the manual attention implementation, but specifying the manual implementati

RuntimeError: "reflection_pad2d" not implemented for 'Half'